## Google earthengine: Hansen et. al
Generate maps via Google EarthEngine Python API with Hansen et. al [Global Forest Change Data] (https://developers.google.com/earth-engine/tutorials/tutorial_forest_02?hl=e) exercise. 

### Initialize earthengine

In [1]:
import ee
import folium

In [2]:
ee.Authenticate()

Enter verification code: 4/1AY0e-g5wBJWcFeHMI-kUeWRCbt4IJd8Xy43dEL244E9dxG9DCUrNjW6nI0Y

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. Here
    self is a folium.Map class instance, and we are adding 
    additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # inherit from parent class
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # create vector feature
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # add vector to Map and return self to allow chaining
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a google earth engine raster layer to a folium map.
        """       
        # handle ImageCollections and Images
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # get the JSON instructions to show image tiles
        map_id_dict = image.getMapId(vis_params)
        
        # get url of the raster tiles
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # create a folium raster layer and add to self
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # add raster to Map and return self to allow chaining
        self.add_child(raster)
        return self

### Hansen et. al Global Forest Change data exercise

In [5]:
# load Hansen data
gfc2014 = ee.Image("UMD/hansen/global_forest_change_2015")

In [6]:
# default settings map
emap = EngineMap()
emap.add_ee_raster(gfc2014, attr="smau8")

In [7]:
# grayscale map (tree cover)
emap.add_ee_raster(gfc2014, 
                   vis_params={"bands": ['treecover2000'],},
                   attr="smau8")

In [8]:
# band types list in current map
gfc2014.bandTypes().getInfo()

{'datamask': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'first_b30': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'first_b40': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'first_b50': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'first_b70': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'gain': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'last_b30': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'last_b40': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'last_b50': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'last_b70': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'loss': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'lossyear': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255},
 'treecover2000': {'type': 'PixelType',
  'precision': 'int',
  'min

In [9]:
# multiple bands
bands1 = gfc2014.visualize(bands=['last_b50', 'last_b40', 'last_b30'], min=0, max=255)
emap.add_ee_raster(bands1, attr="smau8")
emap

In [10]:
# treecover loss/gain, green/binary
bands2 = gfc2014.visualize(bands=['loss', 'treecover2000', 'gain'], min=0, max=255)
emap.add_ee_raster(bands2, attr="smau8")
emap

In [11]:
# treecover loss/gain, varying visualization parameters
bands3 = gfc2014.visualize(bands=['loss', 'treecover2000', 'gain'], min=0, max=[1,255,1])
emap.add_ee_raster(bands3, attr="smau8")
emap

In [12]:
# palette parameters for single band images
palette_emap = EngineMap()
palette = gfc2014.visualize(bands=['treecover2000'], palette=['000000', '00FF00'])
palette_emap.add_ee_raster(palette, attr="palette map")
palette_emap

In [13]:
# palette Paraguay map
palette_emap = EngineMap(location=(-22.0333, -60.6333), zoom_start=9)
palette = gfc2014.visualize(bands=['treecover2000'], palette=['000000', '00FF00'])
palette_emap.add_ee_raster(palette, attr="palette map")
palette_emap

In [14]:
# brighten palette Paraguay map
palette_emap = EngineMap(location=(-22.0333, -60.6333), zoom_start=9)
palette = gfc2014.visualize(bands=['treecover2000'], palette=['000000', '00FF00'], max=100)
palette_emap.add_ee_raster(palette, attr="palette map")
palette_emap

In [15]:
# masking map with itself, shows treecover in green
masking_emap = EngineMap()
treeCover = gfc2014.select(['treecover2000']);
masking_emap.add_ee_raster(treeCover.updateMask(treeCover), 
                           attr="masking map", 
                           vis_params={"palette": ['000000', '00FF00'], 
                                       "max": 100})
masking_emap

In [16]:
# final map
final_map = EngineMap()
treeCover = gfc2014.select(['treecover2000']);
lossImage = gfc2014.select(['loss']);
gainImage = gfc2014.select(['gain']);

#Add the tree cover layer in green.
final_map.add_ee_raster(treeCover.updateMask(treeCover),
                        attr='Forest Cover - smau8',
                        vis_params={"palette": ['000000', '00FF00'], 
                                    "max": 100,
                                   },)

#Add the loss layer in red.
final_map.add_ee_raster(lossImage.updateMask(lossImage),
                        attr='Loss - smau8',
                        vis_params={"palette": ['FF0000']}, );

#Add the gain layer in blue.
final_map.add_ee_raster(gainImage.updateMask(gainImage),
                        attr='Gain - smau8',
                        vis_params={"palette": ['0000FF']});
final_map